In [1]:
#Let connect to the database and print the data from this folder 

import pandas as pd
import os


file_path = r"C:\Users\DeepakSureshNidagund\OneDrive - JA Solar GmbH\Documents - Sales Dashboards (BI Solution)\Y_EU Report\Europe Stock 最新版.xlsx"
    
try:
    # Check if file exists
    if not os.path.exists(file_path):
         raise FileNotFoundError(f"Excel file not found at: {file_path}")
        
    # Read the Excel file
    df = pd.read_excel(file_path, sheet_name="Summary-Europe")
        
    # Print the first few rows of the data
    print("\nFirst 5 rows of the data:")
    print(df.head())
        
except Exception as e:
 print(f"An error occurred: {e}")



First 5 rows of the data:
  DestinationWarehouse Factory Related transaction company  \
0      Rotterdam- KNNL      东台               JA Solar GmbH   
1        Koper- Glovis      奉贤               JA Solar GmbH   
2        Koper- Glovis      奉贤               JA Solar GmbH   
3        Koper- Glovis      奉贤               JA Solar GmbH   
4        Koper- Glovis      奉贤               JA Solar GmbH   

  Related transaction Term        EXW        ETD        ETA update ETA  ATA  \
0                      CIF 2025-05-22 2025-05-27 2025-07-11 2025-07-14  NaN   
1                      CIF 2025-05-27 2025-05-30 2025-07-14        NaT  NaN   
2                      CIF 2025-05-27 2025-05-30 2025-07-14        NaT  NaN   
3                      CIF 2025-05-27 2025-05-30 2025-07-14        NaT  NaN   
4                      CIF 2025-05-27 2025-05-30 2025-07-14        NaT  NaN   

  Date inbound  ...           Type.2 Auxiliary column  SKU  LRF Inv&type 是否签收  \
0          NaT  ...  JAM66D45-620/LB        

In [2]:
df.columns

Index(['DestinationWarehouse', 'Factory', 'Related transaction company',
       'Related transaction Term', 'EXW', 'ETD', 'ETA', 'update ETA', 'ATA',
       'Date inbound', 'inbound status', 'B/L', 'Vessel', 'Sold Date', 'DN',
       'Inv.', 'Type', 'Type.1', 'Container', 'Qty(PC)', 'Qty(plts)',
       'Power(W)', 'Power(w/pc)', 'Status', 'Customer', 'Salesman',
       'Region info', 'Delivery Terms', 'InternalSalesContract',
       'External sales contract', 'Currency', 'Inv No.', 'C2 --> C1 Date',
       'Handover Date', 'Contractual Delivery Week', 'Country Code', '状态',
       'Internal related price', 'Battery type', 'Border Color',
       'Junction box', 'length', 'Voltage', '边框信息', 'Storage duration',
       'Sold（Week）', 'Storage duration（days）', 'original WH',
       'Warehouse after transfer', 'ITS', 'ITS-Remark', 'Identify Risk stock',
       'ERP NO', 'ETD month', 'Sold month', 'outbound quantity',
       'Rest quantity', 'Released on the sea', 'Booking No.', 'EWX Week',
   

In [3]:

# 1. Remove special characters from specified columns
import re
def remove_special_chars(val):
    if pd.isnull(val):
        return val
    # Remove special characters and then strip leading/trailing spaces
    return re.sub(r'[^A-Za-z0-9 ]+', '', str(val)).strip()

columns_to_clean = ['Container', 'DN', 'Inv.']

for col in columns_to_clean:
    if col in df.columns:
        df[col] = df[col].apply(remove_special_chars)

# 2. Rename the columns
df = df.rename(columns={
    'Inv.': 'Invoice Number',
    'Container': 'Container Number',
    'DN': 'Release Number'
})

# 3. Remove specified columns
columns_to_remove = [
    'Factory', 'Related transaction company', 'Related transaction Term', 'Currency', 'Inv No.', 'C2 --> C1 Date',
    'Handover Date', 'Contractual Delivery Week', 'Country Code', '状态', 'Internal related price', 'Battery type',
    'Border Color', 'Junction box', 'length', 'Voltage', 'Storage duration', 'Sold（Week）', 'Storage duration（days）',
    'original WH', 'Warehouse after transfer', 'ETD month', 'Sold month', 'outbound quantity', 'Rest quantity','是否为5.0组件技术(间隙贴膜)'
    'Released on the sea', 'Booking No.', 'EWX Week', 'Type.2', 'Auxiliary column', '成品料号', 'Inv&type', '是否签收',
    '型号', 'Unnamed: 65', 'Unnamed: 66', 'Unnamed: 67','Unnamed: 68', 'Unnamed: 69'
]

# Drop the specified columns
df = df.drop(columns=columns_to_remove, errors='ignore')

# Create a new column 'Ref1' as concatenation of 'Release Number' and 'Container Number'
if 'Release Number' in df.columns and 'Container Number' in df.columns:
    df['Ref1'] = df['Release Number'].astype(str) + df['Container Number'].astype(str)
else:
    df['Ref1'] = None

df.columns


Index(['DestinationWarehouse', 'EXW', 'ETD', 'ETA', 'update ETA', 'ATA',
       'Date inbound', 'inbound status', 'B/L', 'Vessel', 'Sold Date',
       'Release Number', 'Invoice Number', 'Type', 'Type.1',
       'Container Number', 'Qty(PC)', 'Qty(plts)', 'Power(W)', 'Power(w/pc)',
       'Status', 'Customer', 'Salesman', 'Region info', 'Delivery Terms',
       'InternalSalesContract', 'External sales contract', '边框信息', 'ITS',
       'ITS-Remark', 'Identify Risk stock', 'ERP NO', 'Released on the sea',
       'SKU', 'LRF', 'Ref1'],
      dtype='object')

In [4]:
# Separate data into two datasets based on 'Release Number' and 'Not Released'
Released_data = df[(df['Release Number'].notna() & (df['Release Number'].astype(str).str.strip() != "")) & 
         (df['Sold Date'].notna() & (df['Sold Date'].astype(str).str.strip() != ""))]


Not_Released_data = df[(df['Release Number'].isna() | (df['Release Number'].astype(str).str.strip() == "")) & 
         (df['Sold Date'].isna() | (df['Sold Date'].astype(str).str.strip() == ""))]

In [5]:
# Get the latest CDR report
import os
import glob
import datetime
import pandas as pd
from pathlib import Path

# Directory path
reports_dir = r"C:\Users\DeepakSureshNidagund\OneDrive - JA Solar GmbH\Logistics Reporting\000_Master_Query_Reports\Automation_DB\CDR_Reports"

# Check if directory exists
if not os.path.exists(reports_dir):
    print(f"Directory does not exist: {reports_dir}")
    print("Creating directory...")
    os.makedirs(reports_dir, exist_ok=True)

# Try different patterns to find CDR reports
patterns = [
    "CDR_*.csv",  # Primary pattern
    "*CDR*.csv",  # Secondary pattern - any file with CDR in the name
    "*.csv"       # Fallback pattern - any CSV file
]

report_files = []
for pattern in patterns:
    full_pattern = os.path.join(reports_dir, pattern)
    found_files = glob.glob(full_pattern)
    if found_files:
        report_files = found_files
        break

if not report_files:
    print(f"No CSV files found in {reports_dir}")
else:
    try:
        # Try to get the latest file based on date in filename
        latest_file = max(report_files, key=lambda x: 
                        datetime.datetime.strptime(os.path.basename(x).split('_')[1].split('.')[0], 
                                                "%Y-%m-%d"))
    except (IndexError, ValueError):
        print("Could not parse dates from filenames, using file modification time instead.")
        latest_file = max(report_files, key=os.path.getmtime)

    print(f"\nUsing latest report: {os.path.basename(latest_file)}")
    print(f"Full path: {latest_file}")

    try:
        # Read the CSV file and print first 3 rows
        cdr = pd.read_csv(latest_file)
        print("\nFirst 3 rows of the data:")
        print(df.head(3))
    except Exception as e:
        print(f"Error reading file {latest_file}: {e}")




Using latest report: CDR_2025-05-26.csv
Full path: C:\Users\DeepakSureshNidagund\OneDrive - JA Solar GmbH\Logistics Reporting\000_Master_Query_Reports\Automation_DB\CDR_Reports\CDR_2025-05-26.csv

First 3 rows of the data:
  DestinationWarehouse        EXW        ETD        ETA update ETA  ATA  \
0      Rotterdam- KNNL 2025-05-22 2025-05-27 2025-07-11 2025-07-14  NaN   
1        Koper- Glovis 2025-05-27 2025-05-30 2025-07-14        NaT  NaN   
2        Koper- Glovis 2025-05-27 2025-05-30 2025-07-14        NaT  NaN   

  Date inbound inbound status               B/L           Vessel  ...  \
0          NaT            NaN   177SNGNGN89779A  HMM SOUTHAMPTON  ...   
1          NaT            NaN  HLCUSHA2412GOQB8              NaN  ...   
2          NaT            NaN  HLCUSHA2412GOQB8              NaN  ...   

  External sales contract       边框信息  ITS ITS-Remark Identify Risk stock  \
0                     NaN        NaN  NaN        NaN         all regular   
1                     NaN  176

C:\Users\DeepakSureshNidagund\AppData\Local\Temp\ipykernel_21960\3892510382.py:49: DtypeWarning: Columns (12,23,27,28,29,30,31,34,35,36,62,63,64,65,66,67,69,70) have mixed types. Specify dtype option on import or set low_memory=False.
  cdr = pd.read_csv(latest_file)


In [6]:
#Now Combinr the 'Released_data' with CDR data to remove the Outbound data from the Released_data based on Ref1

cdr_columns = ["Ref1", "Current_Status", "Outbound date", "Agreed Delivery date", "Delivery date", "Delivery_Status"]
cdr_selected = cdr[cdr_columns]

# Merge the main dataframe with CDR data using Ref1 as the key
rno = pd.merge(
    Released_data,
    cdr_selected,
    on="Ref1",
    how="left"  # Using left join to keep all records from main df
)




In [7]:
print(rno.columns)

Index(['DestinationWarehouse', 'EXW', 'ETD', 'ETA', 'update ETA', 'ATA',
       'Date inbound', 'inbound status', 'B/L', 'Vessel', 'Sold Date',
       'Release Number', 'Invoice Number', 'Type', 'Type.1',
       'Container Number', 'Qty(PC)', 'Qty(plts)', 'Power(W)', 'Power(w/pc)',
       'Status', 'Customer', 'Salesman', 'Region info', 'Delivery Terms',
       'InternalSalesContract', 'External sales contract', '边框信息', 'ITS',
       'ITS-Remark', 'Identify Risk stock', 'ERP NO', 'Released on the sea',
       'SKU', 'LRF', 'Ref1', 'Current_Status', 'Outbound date',
       'Agreed Delivery date', 'Delivery date', 'Delivery_Status'],
      dtype='object')


In [8]:
#Get the Outbound Pcs from CDR again with summarization by Ref1 and add a new column called Outbound_Pcs_CDR
cdr['Piece'] = pd.to_numeric(cdr['Piece'], errors='coerce')

# Create separate summaries for each status
on_sea_summary = cdr[cdr['Current_Status'] == 'On Sea'].groupby('Ref1')['Piece'].sum().reset_index()
on_sea_summary = on_sea_summary.rename(columns={'Piece': 'On_Sea_Pcs'})

in_stock_summary = cdr[cdr['Current_Status'] == 'In-Stock'].groupby('Ref1')['Piece'].sum().reset_index()
in_stock_summary = in_stock_summary.rename(columns={'Piece': 'In_Stock_Pcs'})

outbounded_summary = cdr[cdr['Current_Status'] == 'Outbounded'].groupby('Ref1')['Piece'].sum().reset_index()
outbounded_summary = outbounded_summary.rename(columns={'Piece': 'Outbounded_Pcs'})

# Merge all summaries with the main dataframe
rno = pd.merge(
    rno,
    on_sea_summary,
    on="Ref1",
    how="left"
)

rno = pd.merge(
    rno,
    in_stock_summary,
    on="Ref1",
    how="left"
)

rno = pd.merge(
    rno,
    outbounded_summary,
    on="Ref1",
    how="left"
)

# Fill NaN values with 0 for all new columns
rno['On_Sea_Pcs'] = rno['On_Sea_Pcs'].fillna(0)
rno['In_Stock_Pcs'] = rno['In_Stock_Pcs'].fillna(0)
rno['Outbounded_Pcs'] = rno['Outbounded_Pcs'].fillna(0)


In [9]:
# Add a new column, Outbound_Pcs_CDR compare which is Outbound_Pcs = Qty(PC) in rno report
rno['Qty(PC)'] = pd.to_numeric(rno['Qty(PC)'], errors='coerce')
rno['Outbounded_Pcs'] = pd.to_numeric(rno['Outbounded_Pcs'], errors='coerce')

# Create comparison column based on Current_Status
import numpy as np
rno['Outbound_Comparison'] = np.where(
    rno['Current_Status'] == 'Outbounded',
    rno['Outbounded_Pcs'] - rno['Qty(PC)'],
    rno['Qty(PC)']
)

#replace the neqtive value of 'Outbound_Comparison' with  rno['Qty(PC)']
rno['Outbound_Comparison'] = np.where(
    rno['Outbound_Comparison'] < 0,
    rno['Qty(PC)'],
    rno['Outbound_Comparison']
)

# Add a new column as Case1 which is ['Outbound_Comparison'] == ['Qty(PC)'] as True and False
rno['Case1'] = np.where(
    rno['Outbound_Comparison'] == rno['Qty(PC)'],
    True,
    False
)

In [10]:
# Get data from WMS Report
import os
import glob
import pandas as pd
from datetime import datetime

#Define the directory path for WMS reports
wms_dir = r"C:\Users\DeepakSureshNidagund\OneDrive - JA Solar GmbH\Documents - Sales Dashboards (BI Solution)\a_Combinded WM_Report\Outbound_WMS_Report"

# Check if directory exists
if not os.path.exists(wms_dir):
    print(f"Error: Directory does not exist: {wms_dir}")
    exit(1)

# Find all Excel files in the directory
excel_files = glob.glob(os.path.join(wms_dir, "*.xlsx"))

# Check if any files were found
if not excel_files:
    print(f"No Excel files found in {wms_dir}")
    exit(1)

# Get the latest file based on modification time
latest_file = max(excel_files, key=os.path.getmtime)

print(f"Found latest WMS report: {os.path.basename(latest_file)}")
print(f"Full path: {latest_file}")
print(f"Last modified: {datetime.fromtimestamp(os.path.getmtime(latest_file))}")

try:
    # Read the Excel file
    wms_Outbound = pd.read_excel(latest_file)
    print(f"\nSuccessfully loaded data with {len( wms_Outbound)} rows and {len( wms_Outbound.columns)} columns")
    
    
    print( wms_Outbound.columns)

except Exception as e:
    print(f"Error reading file {latest_file}: {e}") 
   

Found latest WMS report: Combined_Outbound_WMS_20250522.xlsx
Full path: C:\Users\DeepakSureshNidagund\OneDrive - JA Solar GmbH\Documents - Sales Dashboards (BI Solution)\a_Combinded WM_Report\Outbound_WMS_Report\Combined_Outbound_WMS_20250522.xlsx
Last modified: 2025-05-22 19:16:57.520339

Successfully loaded data with 23954 rows and 26 columns
Index(['Base name', 'Organization Code', 'Organization Name', 'Warehouse',
       'Wharea', 'Wharea Name', 'Wharea.1', 'Internal invoice number',
       'Container number', 'Release number', 'Type', 'Quantity', 'Wattage',
       'Total wattage', 'Trade terms', 'Operation Time', 'statisticsDate',
       'Outbound time', 'Signing time', 'External contract number',
       'Delivery Address', 'Country Code', 'Number of Pallets',
       'carbonFootprint', 'Source_File', 'Ref1'],
      dtype='object')


In [11]:
# # Create Ref1 column as concatenation of Release Number and Container Number
# if 'Release number' in wms_Outbound.columns and 'Container number' in wms_Outbound.columns:
#     wms_Outbound['Ref1'] = wms_Outbound['Release number'].astype(str) + wms_Outbound['Container number'].astype(str)
# else:
#     wms_Outbound['Ref1'] = None

In [12]:
# Based on Ref1 from rno and Ref1 from wms_Outbound, get the total 'Quantity' from wms_Outbound and add it to rno as a new column called 'Outbound_Pcs_WMS'
wms_summary = wms_Outbound.groupby('Ref1')['Quantity'].sum().reset_index()
wms_summary = wms_summary.rename(columns={'Quantity': 'Pcs_from_wms'})

# Merge the summary with rno DataFrame
rno = pd.merge(
    rno,
    wms_summary,
    on='Ref1',
    how='left'  # Use left join to keep all records from rno
)


# Fill any NaN values with 0 for the new column
rno['Pcs_from_wms'] = rno['Pcs_from_wms'].fillna(0)

# Convert to numeric to ensure proper data type
rno['Pcs_from_wms'] = pd.to_numeric(rno['Pcs_from_wms'], errors='coerce')

# Add a new column as Case2 which is rno['Pcs_from_wms'] == rno['Outbounded_Pcs'] as True and False
rno['Case2'] = np.where(
    rno['Pcs_from_wms'] == rno['Outbounded_Pcs'],
    True,
    False
)


# Add a new column as Case3 which is ['Outbound_Comparison'] == ['Qty(PC)'] as True and False
rno['Case3'] = np.where(
    rno['Pcs_from_wms'] == rno['Qty(PC)'] ,
    True,
    False
)

In [13]:
# Get two summaries from CDR:
# 1. Total pieces by container
# 2. Outbounded pieces by container
cdr_total_summary = cdr.groupby('Container No.')['Piece'].sum().reset_index()
cdr_outbound_summary = cdr[cdr['Current_Status'] == 'Outbounded'].groupby('Container No.')['Piece'].sum().reset_index()

# Rename columns to be descriptive
cdr_total_summary = cdr_total_summary.rename(columns={'Piece': 'cnt_Total_Pcs_cdr'})
cdr_outbound_summary = cdr_outbound_summary.rename(columns={'Piece': 'cnt_Outbound_Pcs_cdr'})

# Merge both summaries with rno DataFrame
rno = pd.merge(
    rno,
    cdr_total_summary,
    left_on='Container Number',
    right_on='Container No.',
    how='left'
)

rno = pd.merge(
    rno,
    cdr_outbound_summary,
    left_on='Container Number',
    right_on='Container No.',
    how='left'
)

# Fill NaN values with 0 for both new columns
rno['cnt_Total_Pcs_cdr'] = rno['cnt_Total_Pcs_cdr'].fillna(0)
rno['cnt_Outbound_Pcs_cdr'] = rno['cnt_Outbound_Pcs_cdr'].fillna(0)

# Convert to numeric to ensure proper data type
rno['cnt_Total_Pcs_cdr'] = pd.to_numeric(rno['cnt_Total_Pcs_cdr'], errors='coerce')
rno['cnt_Outbound_Pcs_cdr'] = pd.to_numeric(rno['cnt_Outbound_Pcs_cdr'], errors='coerce')

# Clean up by dropping the extra 'Container No.' columns that were added during merges
rno = rno.drop(['Container No._x', 'Container No._y'], axis=1, errors='ignore')

rno['Case4'] = np.where(
    rno['cnt_Outbound_Pcs_cdr'] == rno['Qty(PC)'],
    True,
    False
)

In [14]:
# Now Add the filters one by one to row the filters

# Now filter out rows that meet all the specified conditions where Currecnt status = Outbounded  & In stock = 0  & Case_2 = True (cdr pcs = wms pcs only outbound )
rno = rno[~((rno['Current_Status'] == 'Outbounded') & 
            (rno['In_Stock_Pcs'] == 0) & 
            (rno['Case2'] == True))]


# Now filter out rows that meet all the specified conditions where Currecnt status = Outbounded  & In stock = 0  & Case_3 = True (wms pcs = wms pcs only outbound )
rno = rno[~((rno['Current_Status'] == 'Outbounded') & 
            (rno['In_Stock_Pcs'] == 0) & 
            (rno['Case3'] == True))]



# Now filter out rows that meet all the specified conditions where Currecnt status = Outbounded  & In stock = 0  & Case_4 = True (cdr pcs = rno pcs at cotainer lvl pcs only outbound )
rno = rno[~((rno['Current_Status'] == 'Outbounded') & 
            (rno['In_Stock_Pcs'] == 0) & 
            (rno['Case4'] == True))]

# Remove the row where the Current status == Outbounded and Outbound_Comparison ==0
rno = rno[
    ~((rno['Current_Status'] == 'Outbounded') & 
      (rno['Outbound_Comparison'] == 0))
]


# Now remove the row where the Current status == Outbounded and Case_2 == TRUE
rno = rno[
    ~((rno['Current_Status'] == 'Outbounded') & 
      (rno['Case2'] == True))
]

# Remove the rows Current status == Outbounded
rno = rno[
    ~((rno['Current_Status'] == 'Outbounded'))]


# Remove rows where Container Number is null/blank
rno = rno[rno['Container Number'].notna() & (rno['Container Number'].str.strip() != '')]


# Read the Remove_data file
remove_data_path = r"C:\Users\DeepakSureshNidagund\Downloads\Reporting Application\Automation\automation\tests\Remove_data.xlsx"
remove_df = pd.read_excel(remove_data_path)
ref1_to_remove = remove_df['Ref1'].tolist()
rno = rno[~rno['Ref1'].isin(ref1_to_remove)]

# Reorder columns to bring specified columns to front
first_columns = ['Ref1', 'Container Number', 'Release Number']
other_columns = [col for col in rno.columns if col not in first_columns]
rno = rno[first_columns + other_columns]

# change these columns to date format
rno['Sold Date'] = pd.to_datetime(rno['Sold Date'])
rno['Outbound date'] = pd.to_datetime(rno['Outbound date'])
rno['Agreed Delivery date'] = pd.to_datetime(rno['Agreed Delivery date'])
rno['Delivery date'] = pd.to_datetime(rno['Delivery date'])





In [16]:
import openpyxl

# Define the file path
excel_path = r"C:\Users\DeepakSureshNidagund\OneDrive - JA Solar GmbH\Logistics Reporting\000_Master_Query_Reports\Automation_DB\RNO_Report\RNO_Report - Copy.xlsx"

# Load the existing workbook
book = openpyxl.load_workbook(excel_path)

# Select the sheet
sheet = book['RNO Report']

# Delete all rows except the header
while sheet.max_row > 1:  # Keep the first row (header)
    sheet.delete_rows(2)  # Always delete row 2 until only header remains

# Save the workbook after clearing data
book.save(excel_path)
book.close()

# Write the new data
with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    rno.to_excel(writer, sheet_name='RNO Report', index=False, header=False, startrow=1)

In [41]:
# Released_data.to_csv("Released_data.csv", index=False)
# Not_Released_data.to_csv("Not_Released_data.csv", index=False)